In [10]:
import pandas as pd
import datetime as dt
import numpy as np

In [11]:
ema = pd.read_csv("/home/donsson/proyectos/MODELO ABASTECIMIENTO/csvsalidas/ema_mio202538.csv")

In [12]:
ema = ema.copy()

# -------------------------------
# 1. Calcular venta_costo_ema
# -------------------------------
ema["venta_costo_ema"] = (ema["EMA"] * ema["producto_costo_unitario"]).round(2)

# -------------------------------
# 2. Totales por sucursal
# -------------------------------
ema["venta_costo_tot"] = ema.groupby("store_name")["venta_costo_ema"].transform("sum")
ema["total_ema"] = ema.groupby("store_name")["EMA"].transform("sum")

# -------------------------------
# 3. Proporciones dentro de cada sucursal
# -------------------------------
ema["venta_costo%"] = (ema["venta_costo_ema"] / ema["venta_costo_tot"]).round(4)
ema["cantidad%"] = (ema["EMA"] / ema["total_ema"]).round(4)

# -------------------------------
# 4. Orden y acumulados
# -------------------------------
# Ordenar por costo dentro de cada sucursal
ema = ema.sort_values(["store_name", "venta_costo_ema"], ascending=[True, False])
ema["acumulado_costo"] = ema.groupby("store_name")["venta_costo%"].cumsum().round(4)

# Ordenar por cantidad dentro de cada sucursal
ema = ema.sort_values(["store_name", "EMA"], ascending=[True, False])
ema["acumulado_cantidad"] = ema.groupby("store_name")["cantidad%"].cumsum().round(2)

# Acumulado de desviación por sucursal
if "desviacion_ema%" in ema.columns:
    ema["acumulado_desviacion"] = (
        ema.groupby("store_name")["desviacion_ema%"].cumsum().round(2)
    )

# -------------------------------
# 5. Renombrar columnas (estilo reporte Odoo)
# -------------------------------
ema_def = ema.rename(
    columns={
        "año": "Año",
        "semana_num": "semana",
        "EMA": "Ema",
        "producto_costo_unitario": "Costo unitario",
        "venta_costo_ema": "Venta costo EMA",
        "venta_costo%": "Venta costo %",
        "acumulado_costo": "Acumulado costo",
        "desviacion_ema%": "Desviacion EMA %",
    }
)

# -------------------------------
# 6. Calcular desviación en dinero
# -------------------------------
if "Desviacion EMA %" in ema_def.columns:
    ema_def["Desviacion EMA"] = (
        ema_def["Venta costo EMA"] * ema_def["Desviacion EMA %"]
    ).round(1)



# Ejemplo de reglas (lo que viene de conf_acumulado_desviacion en Odoo) #PERILLA
reglas = [
    {"rango_ini": 0, "rango_fin": 0.8, "valor": 0},
    {"rango_ini": 0.8, "rango_fin": 1.5, "valor": 0.5},
    {"rango_ini": 1.5, "rango_fin": 3, "valor": 0.8},
    {"rango_ini": 3, "rango_fin": 2000, "valor": 1},
]

# aplicar reglas 0–0.8–1 como antes, pero sobre coef_ventas_norm


# Función que asigna el valor según reglas
def asignar_desviacion(coef, reglas):
    for r in reglas:
        if r["rango_ini"] <= coef <= r["rango_fin"]:
            return r["valor"]
    return np.nan  # si no entra en ningún rango

# Aplicar
ema_def["acumulado_desviacion_p"] = ema_def["coef_ventas"].apply(
    lambda x: asignar_desviacion(x, reglas)
)



# -------------------------------
# 7. Limpiar columnas innecesarias
# -------------------------------
if "Unnamed: 0" in ema_def.columns:
    ema_def = ema_def.drop(columns=["Unnamed: 0"])


# 8. Reglas acumulado costo combinado
# --- Configuración ---
def check_div(a, b):
    try:
        return a / b if b != 0 else 0
    except Exception:
        return 0

## PERILLA

conf_porcentajes_impacto = {
    "cantidad": 0.5,
    "costo": 0.3,
    "desviacion": 0.2,
    "cantidad_mayor": 0.7,
    "costo_mayor": 0.3,
}

# Valor de referencia en porcentaje (ejemplo: 10%) ##PERILLA
conf_acumulado_cantidad = 80
conf_acumulado_cantidad_desviacion = check_div(conf_acumulado_cantidad, 100)


# --- Cálculo en el DataFrame ---
def calcular_combinado(row):
    acumulado_cantidad_final = round(row["acumulado_cantidad"], 2)
    acumulado_costo_final = round(row["Acumulado costo"], 2)
    acumulado_desviacion_p_final = round(row["acumulado_desviacion_p"], 2)

    if acumulado_cantidad_final >= conf_acumulado_cantidad_desviacion:
        return (
            acumulado_cantidad_final * conf_porcentajes_impacto["cantidad_mayor"]
            + acumulado_costo_final * conf_porcentajes_impacto["costo_mayor"]
        )
    else:
        return (
            acumulado_cantidad_final * conf_porcentajes_impacto["cantidad"]
            + acumulado_costo_final * conf_porcentajes_impacto["costo"]
            + acumulado_desviacion_p_final * conf_porcentajes_impacto["desviacion"]
        )

# Nueva columna en tu df
ema_def["acumulado_combinado"] = ema_def.apply(calcular_combinado, axis=1).round(2)


def clasificar_pareto(valor):
    if valor <= 0.50:
        return "AAA"
    elif valor < 0.8:
        return "A"
    elif valor < 0.95:
        return "B"
    else:
        return "C"

ema_def["Clasificacion"] = ema_def["acumulado_combinado"].apply(clasificar_pareto)


In [13]:
ema_ord = ema_def[["store_name","product_ref","Año","semana","Ema",
"Costo unitario","Venta costo EMA","Venta costo %","Acumulado costo","Desviacion EMA",
"Desviacion EMA %","acumulado_cantidad","coef_ventas","acumulado_desviacion","acumulado_desviacion_p","acumulado_combinado","Clasificacion"]]

ema_ord.to_csv("/home/donsson/proyectos/MODELO ABASTECIMIENTO/csvsalidas/clasificacion_mia2025.csv")

bq = ema_ord[ema_ord["store_name"]=="SUCURSAL BARRANQUILLA"].sort_values(by=["Ema"],ascending=False)

In [14]:
bq.head((5))

,store_name,product_ref,Año,semana,Ema,Costo unitario,Venta costo EMA,Venta costo %,Acumulado costo,Desviacion EMA,Desviacion EMA %,acumulado_cantidad,coef_ventas,acumulado_desviacion,acumulado_desviacion_p,acumulado_combinado,Clasificacion
15277,SUCURSAL BARRANQUILLA,DAB02570025,2025.0,38,42.844029,13507.71,578724.72,0.0091,0.2129,121532.2,0.21,0.02,4.151229,0.21,1.0,0.27,AAA
18049,SUCURSAL BARRANQUILLA,DAB14570025,2025.0,38,37.895238,9774.98,370425.19,0.0058,0.2963,218550.9,0.59,0.04,5.238390,0.80,1.0,0.31,AAA
6566,SUCURSAL BARRANQUILLA,BCS00035125,2025.0,38,37.128322,20638.72,766281.04,0.0120,0.1194,114942.2,0.15,0.06,2.033898,0.95,0.8,0.23,AAA
12786,SUCURSAL BARRANQUILLA,BLS00037125,2025.0,38,31.785192,32797.97,1042489.77,0.0163,0.0824,104249.0,0.10,0.08,1.564045,1.05,0.8,0.22,AAA
15453,SUCURSAL BARRANQUILLA,DAB02772025,2025.0,38,30.777496,10680.97,328733.51,0.0051,0.3335,62459.4,0.19,0.09,3.207127,1.24,1.0,0.34,AAA


In [15]:
df_real = pd.read_csv("/home/donsson/proyectos/API/real_ema.csv")

In [16]:
df_real= df_real[["store_name","almacen_name","product_name","semana","ema","acumulado_combinado","clasificacion"]]
df_mio = ema_ord[["store_name","product_ref","semana","Ema","acumulado_combinado","Clasificacion"]]


In [17]:
conteo = pd.crosstab(df_mio["store_name"], df_mio["Clasificacion"])
conteo

Clasificacion,A,AAA,B,C
store_name,,,,
0,0,0,0,566
PRINCIPAL COTA,149,28,238,3894
SUCURSAL BARRANQUILLA,216,45,334,3714
SUCURSAL BUCARAMANGA,200,46,309,3754
SUCURSAL CALI,180,38,294,3797
SUCURSAL CALLE 6,184,58,369,3698
SUCURSAL MEDELLIN,141,35,283,3850
SUCURSAL NORTE,159,33,285,3832
SUCURSAL VALLADOLID,134,29,274,3872


In [18]:
contar = pd.crosstab(df_real["almacen_name"], df_real["clasificacion"])
contar

clasificacion,A,AAA,B,C
almacen_name,,,,
ANTIOQUEÑA DE LUBRICANTES SGP SAS,24,5,37,4796
BARRANQUILLA,210,41,384,4227
BUCARAMANGA,208,39,319,4296
CALI,153,28,291,4390
CALLE 6,199,55,366,4242
MEDELLIN,143,34,314,4371
MOSTRADOR COTA,119,18,281,4444
NORTE,168,26,298,4370
VALLADOLID,149,29,280,4404
